In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/bankloan_dataset_prep.csv")
#model은 정제화된 csv를 읽어야하기 때문에 EDA와 전처리 과정을 거치는 것임. 
#너무 많아서 그런게 아니고, 정확성을 높이기 위해 이런 과정을 거친다.
df.head(3)

,Unnamed: 0,Unnamed: 0.1,CUSTOMER_ID,LOAN_YEAR,LOAN_DT,WORK_TERM,HOME_OWNERSHIP,INCOME_LEVEL,YEAR_INCOME,LOAN_AMOUNT,...,CUSTOMER_LEVEL,DTI,LAST_PAYMENT,TOTAL_RECV_PRINCIPAL,RECV_LOAN,INSTALLMENT_PAYMENT,AREA,CUSTOMER_GRADE,HOME_OWNERSHIP_LE,CUSTOMER_GRADE_LE
0,0,0,231321,2008,2008-01-01,5.0,NONE,Low,22200,10000,...,A,52.03,11239.56,9999.99,0.0,312.22,leinster,E,2,4
1,1,1,79893,2007,2007-01-07,0.5,NONE,Low,75027,6700,...,A,10.31,7530.42,6700.00,0.0,209.18,Northern-Irl,B,2,1
2,2,2,79924,2007,2007-01-08,0.5,NONE,Low,75027,7000,...,A,10.78,7867.53,7000.00,0.0,218.55,Northern-Irl,B,2,1


In [32]:
features = ['YEAR_INCOME', 'LOAN_AMOUNT', 'LOAN_RATE']
X = df[features]
X.head(3)

,YEAR_INCOME,LOAN_AMOUNT,LOAN_RATE
0,24000,5000,10.65
1,30000,2500,15.27
2,12252,2400,15.96


In [49]:
y = df['DTI']

KeyError: 'DTI'

In [50]:
y.head(3)

0    25.27
1    10.88
2    25.84
Name: DTI, dtype: float64

In [34]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(X, y, random_state=1)


In [35]:
#의사 결정 트리.

from sklearn.tree import DecisionTreeRegressor
dmodel = DecisionTreeRegressor(random_state=1)
dmodel.fit(train_input,train_target)

DecisionTreeRegressor(random_state=1)

In [36]:
print("TRAIN : ", dmodel.score(train_input,train_target))
print("TEST : ", dmodel.score(test_input,test_target))

TRAIN :  1.0
TEST :  0.9601380753951912


In [37]:
print(y)

0         25.27
1         10.88
2         25.84
3         25.81
4          4.70
          ...  
768868    35.55
768869    31.52
768870    22.87
768871    36.96
768872    53.06
Name: DTI, Length: 768873, dtype: float64


In [38]:
from xgboost import XGBRegressor
xgb_model = XGBRegressor(n_estimators=20, max_depth=10, random_state=1)
xgb_model.fit(train_input, train_target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=20, n_jobs=4, num_parallel_tree=1, random_state=1,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [39]:
#의사결정트리
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=1)
#random_state=1을 하면 값이 계속 바뀜, 스코어가 바뀜 >> random_state가 뭔지 찾아보기

In [40]:
model.fit(X,y)

DecisionTreeRegressor(random_state=1)

In [41]:
model.score(X,y)
#score가 몇일때 유의미한 데이터인가? 

1.0

In [42]:
import joblib
joblib.dump(model,'./model/dti_model.pkl')

['./model/dti_model.pkl']

In [43]:
#depth줄인버전
import joblib
joblib.dump(xgb_model,'./model/dti_model_xgboost2.pkl')

['./model/dti_model_xgboost2.pkl']

In [44]:
model = joblib.load('./model/dti_model_xgboost2.pkl')

In [51]:
매년수입 = 24000
대출액 = 5000
대출이자 = 10.65

df = pd.DataFrame([[매년수입, 대출액, 대출이자]],
                 columns=['YEAR_INCOME', 'LOAN_AMOUNT', 'LOAN_RATE'])

print(df)

result = model.predict(df)
print('\n')
print('DTI(총부채상환비율) : ', result[0], "%")

   YEAR_INCOME  LOAN_AMOUNT  LOAN_RATE
0        24000         5000      10.65


DTI(총부채상환비율) :  24.971647 %
